In [ ]:
import mxnet as mx
import numpy as np

In [ ]:
prefix="cnn_models/resnet20_clustered8"
epoch=0
sym, args, auxs = mx.mod.module.load_checkpoint(prefix, epoch)

In [ ]:
def extract_cluster_centers(data):
    shape = data.shape
    nclusters = shape[0]/8
    print shape
    q_indices=np.zeros((shape[0],shape[1]))
    codewords=np.zeros((nclusters, shape[1], shape[2], shape[3]))
    
    for channel in range(shape[1]):
        cluster_centers, reconstructed_indices=np.unique(data[:,channel,:,:].asnumpy(), return_inverse=True, axis=0)
        codewords[:,channel,:,:] = cluster_centers
        q_indices[:,channel] =reconstructed_indices
        
    return  codewords, q_indices

In [ ]:
def process_symbol(in_sym, in_args, in_shape):
    layers = []
    network = {}
    filter_codebook={}
    for element in in_sym.get_internals().list_outputs():
         if "conv" in element and "weight" in element: #maybe tune for different networks
                if "conv0" in element: #hack to exclude first layer
                    continue
                weight=in_args[element]
                layer=element[:len(element)-7]
                
                codebook_filter, indices = extract_cluster_centers(weight)
                lrshape=in_sym.get_internals()[layer+"_output"].infer_shape(data=in_shape)[1]
                
                layers.append(layer)
                network[layer]={}
                filter_codebook[layer] = weight
                network[layer]["indices"] = indices
                network[layer]["f_shape"]=weight.shape
                network[layer]["c_shape"]=codebook_filter.shape
                network[layer]["out_shape"] = lrshape
                
    return network
    
    
net_info = process_symbol(sym,args,(7,3,32,32))    
    

In [ ]:
net_info

In [ ]:
sym.get_internals()['stage3_unit1_conv1_output'].infer_shape(data=(7,3,32,32))[1]